<a href="https://colab.research.google.com/github/barbarajunq131/Programas/blob/main/Exercicio__Lista_1__Modelagem_estatistica_B%C3%A1rbaraJunqueira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Disciplina : Modelagem Estatística - T01 (2025.2) /n

Aluna: Bárbara Junqueira dos Santos /n

Matricula: 20252001826 /n

Lista de exercício: 1/n

link do colab: https://colab.research.google.com/drive/10vrVH6Hrp3OzfKf5b_jF-Lw-pJ5ocEjb#scrollTo=PQXmsPBUL6z9

1. Bibliotecas

In [42]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.sandwich_covariance import cov_hc0
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib as mpl
import textwrap

PATH_BANCO = '/content/banco.csv'
OUTPUT_PDF = '/content/lista1_report.pdf'
MAX_PAGES = 10
FONT_NAME = 'Calibri'
FONT_SIZE = 11

mpl.rcParams['font.family'] = FONT_NAME
mpl.rcParams['font.size'] = FONT_SIZE

import textwrap

2. Carregar arquivo e pré-tratamentos


In [43]:
def wrap_lines(text, width=95):
  return textwrap.wrap(text, width)

def page_text(pdf, title, lines):
  fig, ax = plt.subplots(figsize=(8.27, 11.69)) # A4
  ax.axis('off')
  y = 0.95
  ax.text(0.02, y, title, fontsize=12, weight='bold', transform=fig.transFigure)
  y -= 0.04
  for line in lines:
      wrapped = wrap_lines(line, width=95)
      for sub in wrapped:
          ax.text(0.02, y, sub, fontsize=FONT_SIZE, transform=fig.transFigure)
          y -= 0.03
          if y < 0.05:
            break
      if y < 0.05:
        break
  pdf.savefig(fig)
  plt.close(fig)

print('Carregando dados de', PATH_BANCO)
try:
  banco = pd.read_csv(PATH_BANCO)
  print('Banco carregado:', banco.shape[0], 'linhas,', banco.shape[1], 'colunas')
except Exception as e:
  raise SystemExit('Erro ao carregar banco.csv. Faça upload para /content e ajuste PATH_BANCO. Erro: ' + str(e))

Carregando dados de /content/banco.csv
Banco carregado: 1998 linhas, 252 colunas


In [44]:
if 'instrucao' in banco.columns:
  banco['instrucao'] = banco['instrucao'].astype('category')
if 'ramo' in banco.columns:
  banco['ramo'] = banco['ramo'].astype('category')

In [45]:
formula_prof = '''
logsalh ~ idade + I(idade**2) + mulher
+ C(instrucao, Treatment(reference=1))
+ C(ramo, Treatment(reference=1))
+ rm + nao_branco + urbano + form
'''

3. Questão 1 – Regressão Simples: Educação e Salário

In [46]:
modelo_q1 = smf.ols('logsalh ~ C(instrucao)', data=banco).fit()
q1_lines = [
  'Modelo: logsalh ~ C(instrucao)',
  'Referência: instrucao = 1',
  'Coeficientes:'
]
for name, coef in modelo_q1.params.items():
  p = modelo_q1.pvalues[name]
  signif = 'p<0.05' if p < 0.05 else f'p={p:.3f}'
  q1_lines.append(f'{name}: coef={coef:.4f}, {signif}')
q1_lines.append('Interpretação: coef em log ≈ diferença percentual em relação à categoria base.')


4. Questão 2 – Regressão Múltipla: Controlando por Outras Variáveis

In [47]:
modelo_q2 = smf.ols(formula=formula_prof, data=banco).fit()
q2_lines = [
  'Modelo completo conforme arquivo do professor.',
  'Coeficientes chave:'
]
for v in ['idade','I(idade ** 2)','mulher','rm','nao_branco','urbano','form']:
  if v in modelo_q2.params.index:
      q2_lines.append(f'{v}: coef={modelo_q2.params[v]:.4f}, p={modelo_q2.pvalues[v]:.4f}')
q2_lines.append('Interpretação: controle reduz viés de variáveis omitidas; coeficientes mudam.')

5. Questão 3 – Experiência e Ciclo de Vida

In [48]:
b1 = modelo_q2.params.get('idade', np.nan)
b2 = modelo_q2.params.get('I(idade ** 2)', np.nan)
q3_lines = [f'Coef idade={b1:.6f}, coef idade^2={b2:.6f}']
if b2 != 0:
  idade_max = -b1/(2*b2)
  q3_lines.append(f'Idade de maior rendimento ≈ {idade_max:.1f} anos.')
q3_lines.append('Interpretação: forma quadrática sugere crescimento e queda posteriores.')

6. Questão 4 – Testando Heterocedasticidadeerpretação dos resultados

In [49]:
lm_stat, lm_pvalue, f_stat, f_pvalue = het_breuschpagan(modelo_q2.resid, modelo_q2.model.exog)
q4_lines = [
  f'LM={lm_stat:.3f}, p-valor={lm_pvalue:.4f}',
  'H0: homocedasticidade. p<0.05 ⇒ heterocedasticidade.'
]

7. Questão 5 – Erros-Padrão Robustos

In [50]:
modelo_q5 = smf.ols(formula=formula_prof, data=banco).fit(cov_type='HC0')

q5_lines = ['Comparação SE tradicionais vs robustos:']
for name in ['Intercept', 'idade', 'mulher', 'rm', 'nao_branco', 'urbano', 'form']:
    if name in modelo_q2.params.index:
        se_trad = modelo_q2.bse[name]
        se_rob = modelo_q5.bse[name]
        sig_trad = 'SIGNIF' if modelo_q2.pvalues[name] < 0.05 else 'NÃO SIGNIF'
        sig_rob = 'SIGNIF' if modelo_q5.pvalues[name] < 0.05 else 'NÃO SIGNIF'
        q5_lines.append(f'{name}: SE trad={se_trad:.4f} ({sig_trad}), SE rob={se_rob:.4f} ({sig_rob})')

q5_lines.append('Interpretação: SE robustos corrigem heterocedasticidade e podem alterar significância.')


8. Questão 6 – Previsão de Salário-Hora

In [51]:
novo = pd.DataFrame({
  'idade':[30],
  'mulher':[1],
  'instrucao':pd.Categorical([6], categories=banco['instrucao'].cat.categories) if 'instrucao' in banco.columns else pd.Categorical([6]),
  'ramo':pd.Categorical([8], categories=banco['ramo'].cat.categories) if 'ramo' in banco.columns else pd.Categorical([8]),
  'rm':[1],
  'nao_branco':[1],
  'urbano':[1],
  'form':[1]
})


pred_log = modelo_completo.predict(novo)[0]
pred_salh = np.exp(pred_log)
mean_salh = np.exp(banco['logsalh']).mean()
q6_lines = [
  'Código (essencial): modelo_completo.predict(novo)',
  'Características: 30 anos, mulher, instrucao=6, ramo=8, rm=1, nao_branco=1, urbano=1, form=1',
  f'Log-salário previsto = {pred_log:.4f}',
  f'Salário-hora previsto = {pred_salh:.4f}',
  f'Média amostral do salário-hora = {mean_salh:.4f}',
  'Interpretação curta: comparar previsão com média para avaliar posição relativa.'
]

In [ ]:
Apêndice

In [52]:
ap_lines = ['Coeficientes completos (curto):']
for name, coef in modelo_q2.params.items():
    ap_lines.append(f'{name}: coef={coef:.4f}, p={modelo_q2.pvalues[name]:.4f}')

In [41]:
pages = [
    ('Capa', ['Lista 1 – Modelagem Estatística', 'Relatório automático', 'Questões 1 a 6 + Apêndice']),
    ('Questão 1', q1_lines),
    ('Questão 2', q2_lines),
    ('Questão 3', q3_lines),
    ('Questão 4', q4_lines),
    ('Questão 5', q5_lines),
    ('Questão 6', q6_lines),
    ('Apêndice', ap_lines)
]

if len(pages) > MAX_PAGES:
    raise SystemExit(f'PDF teria {len(pages)} páginas (>10). Reduza conteúdo.')

with PdfPages(OUTPUT_PDF) as pdf:
    for title, content in pages:
        page_text(pdf, title, content)

print('PDF final gerado em:', OUTPUT_PDF)

PDF final gerado em: /content/lista1_report.pdf
